# Model Evaluation
* What we've done so far to evaluate our supervised models:
 * split our dataset into a training set and a test: __`train_test_split()`__
 * built a model on the training set: __`fit()`__
 * evaluated the model it on the test set: __`score()`__

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# create a synthetic dataset
X, y = make_blobs(random_state=0)
print(X, y)
# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# instantiate a model and fit it to the training set
logreg = LogisticRegression().fit(X_train, y_train)
# evaluate the model on the test set
print("Test set score: {:.2f}".format(logreg.score(X_test, y_test)))

# What are we actually interested in measuring?
* how well our model generalizes to new, unseen data
* (...NOT how well our model fit the training data)

# Cross Validation
* a statistical method of evaluating generalization performance
* more stable and thorough than using a split into a training and a test set
* instead, data is split repeatedly and multiple models are trained
* most common method of cross-validation is _k-fold_ cross-validation (k = 5 or 10)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
iris = load_iris()
logreg = LogisticRegression(solver='liblinear')
scores = cross_val_score(logreg, iris.data, iris.target)
print("Cross-validation scores: {}".format(scores))

* How many folds were there by default?
* Let's increase that to see if we do better...

In [ ]:
scores = cross_val_score(logreg, iris.data, iris.target, cv=5)
print("Cross-validation scores: {}".format(scores))

In [ ]:
print("Average cross-validation score: {:.2f}".format(scores.mean()))

* we can conclude that we expect the model to be around 96% accurate on average
* relatively high variance in the accuracy between folds, ranging from 90-100%
 * model could be very dependent on the particular folds used for training
 * could also just be a consequence of the small size of the dataset

# Uh-Oh!

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
print("Iris labels:\n{}".format(iris.target))

* If the data is structured in a certain way (like this) we can have major variation between the folds.
* Even with the folding model, the data structure itself can cause issues.

# Stratified Cross Validation

* Split the data such that the proportions between classes are the same in each fold as they are in the whole dataset
![alt-text](images/cv-2.png)

# KFold Validation

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)

In [ ]:
print("Cross-validation scores:\n{}".format(
    cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

In [ ]:
from collections import Counter

def show_fold_contents(kf):
    for fold in kf.split(iris.data):
        c = Counter()
        for datapoint in fold[0]:
            c.update([iris.target_names[iris.target[datapoint]]])
        print(c)
        
show_fold_contents(kfold)

In [ ]:
kfold = KFold(n_splits=3)
print("Cross-validation scores:\n{}".format(
    cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

In [ ]:
show_fold_contents(kfold)

* What happened between 3 and 5 splits?

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("Cross-validation scores:\n{}".format(
    cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

# Leave One Out Validation

In [ ]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(logreg, iris.data, iris.target, cv=loo)
print("Number of cv iterations: ", len(scores))
print("Mean accuracy: {:.2f}".format(scores.mean()))

* Another type of cross-fold
* Train on everything but one sample, rotate and do it again, leaving out a different sample.
* What is the tradeoff?

# Shuffle Split Cross Validation

![alt-text](images/cv-3.png)
* Combine shuffle with split with cross validation
* The data blender is on "High" now

In [ ]:
from sklearn.model_selection import ShuffleSplit
shuffle_split = ShuffleSplit(test_size=.25, train_size=.75, n_splits=10)
scores = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)
print('Cross-validation scores:\n', scores)

# GroupKFold Validation

* suppose we want to build a system to recognize emotions from pictures of faces

![alt-text](images/emotions.jpg)

* we collect a dataset of pictures of 100 people
 * each person is captured multiple times, showing various emotions
* goal is to build a classifier that correctly identifies emotions of people not in the dataset
* you could use the default stratified cross-validation to measure the performance of a classifier
* however, it's likely that pictures of the same person will be in both training and test set
* much easier for a classifier to detect emotions in a face that is part of the training set, compared to a completely new face
* in order to accurately evaluate generalization to new faces, we must ensure that  training and test sets contain images of different people
* to do this, we can use GroupKFold, which takes an array of groups as argument that we can use to indicate which person is in the image
* the groups array here indicates groups in the data that should not be split when creating the training and test sets, and should not be confused with the class label
![alt-text](images/cv-4.png)

In [ ]:
from sklearn.model_selection import GroupKFold
# create synthetic dataset
X, y = make_blobs(n_samples=12, random_state=0)
# assume the first three samples belong to the same group,
# then the next four, etc.
scores = cross_val_score(logreg, X, y, groups=[0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3], cv=GroupKFold(n_splits=3))
print("Cross-validation scores:\n", scores)

# Model Performance - Linear Classifiers
![alt-text](images/svc.png)

# Regularization Parameters
* Do we care more about generalizing, or getting things right?
* Support Vector Machines have hyperparameters C and gamma
* C = _cost_ for making errors
  * A large C gives you low bias and high variance
  * A small C gives you higher bias and lower variance

![alt-text](images/svc-1.png)

### gamma = kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
 * higher values of gamma will result in exact fit as per training data set, i.e., generalization error and overfitting
 * gamma control how "pointy" the peaks are–low gamma = smooth, higher gamma = pointy

![alt-text](images/gamma.png)

In [ ]:
# naive grid search implementation
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(
                    iris.data, iris.target, random_state=0)
print("Size of training set: {}   size of test set: {}".format(
         X_train.shape[0], X_test.shape[0]))

best_score = 0

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # for each combination of parameters, train an SVC
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        # evaluate the SVC on the test set
        score = svm.score(X_test, y_test)
        # if we got a better score, store the score and parameters
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}

print("Best score: {:.2f}".format(best_score))
print("Best parameters: {}".format(best_parameters))

# What's the problem with the above?

# Train-Validate-Test

![alt-text](images/train-validate-test.png)
### Split the data 3 ways now....
* Training (to train the model on)
* Validation (to refine hyperparameters on)
* Test (to actually test)

In [ ]:
from sklearn.svm import SVC
# split data into train+validation set and test set
X_trainval, X_test, y_trainval, y_test = train_test_split(
    iris.data, iris.target, random_state=0)

In [ ]:
# split train+validation set into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
    X_trainval, y_trainval, random_state=1)
len(X_train)

In [ ]:
best_score = 0
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # for each combination of parameters train an SVC
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        # evaluate the SVC on the validation set
        score = svm.score(X_valid, y_valid)
        # if we got a better score, store the score and parameters
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}
            
print(best_score, best_parameters)

### A note about ** the syntax below
* __`best_parameters`__ is a dict(ionary), i.e., a group of key/value pairs
* the __`SVC`__ method accepts _keyword arguments_, meaning instead of just passing the args directly, they have to be passed as __`key=value`__ arguments
* prefacing a dict with __`**`__ causes the key/value pairs in the dict to be "exploded" into __`key=value`__ arguments

In [ ]:
print("Size of training set: {} size of validation set: {} size of test set: {}".format(
         X_train.shape[0], X_valid.shape[0], X_test.shape[0]))
# rebuild a model on the combined training and validation set,
# and evaluate it on the test set
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)
test_score = svm.score(X_test, y_test)
print("Best score on validation set: {:.2f}".format(best_score))
print("Best parameters: ", best_parameters)
print("Test set score with best parameters: {:.2f}".format(test_score))

# Cross Validation and Grid Search

In [ ]:
import numpy as np
# reference: manual_grid_search_cv
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # for each combination of parameters,
        # train an SVC
        svm = SVC(gamma=gamma, C=C)
        # perform cross-validation
        scores = cross_val_score(svm, X_trainval, y_trainval, cv=5)
        # compute mean cross-validation accuracy
        score = np.mean(scores)
        # if we got a better score, store the score and parameters
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}
# rebuild a model on the combined training and validation set
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)
print(best_parameters, best_score)
        

# GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

grid_search = GridSearchCV(SVC(),
                param_grid, cv=5, return_train_score=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
     iris.data, iris.target, random_state=0)
grid_search.fit(X_train, y_train)

In [ ]:
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

In [ ]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

In [ ]:
print("Best estimator:\n{}".format(grid_search.best_estimator_))